# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.   

Описание данных:   

Данные геологоразведки трёх регионов находятся в файлах:   
/datasets/geo_data_0.csv   
/datasets/geo_data_1.csv   
/datasets/geo_data_2.csv   
id — уникальный идентификатор скважины;   
f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);   
product — объём запасов в скважине (тыс. баррелей).   

## Загрузка и подготовка данных

In [35]:
import pandas as pd
import traceback
import os.path
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
import plotly.express as px
import matplotlib.pyplot as plt
from numpy.random import RandomState
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 29)
data_files = {'Регион_0': pd.Series(['geo_data_0.csv'], index = ['file_name']), 
              'Регион_1':pd.Series(['geo_data_1.csv'], index = ['file_name']), 
              'Регион_2': pd.Series(['geo_data_2.csv'], index = ['file_name'])
             }
local_path = os.path.join('C:', os.sep, 'Users', 'timpu', 'Desktop', 'Data Science(Проекты и шпаргалки)',
                          'Проекты', '8 Нефтедобыча', 'data')
web_dir_path = os.path.join(os.sep, 'datasets' + os.sep)
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [36]:
for key_name, reg_data in data_files.items():
    try:
        reg_data['data']  = pd.read_csv(web_dir_path + reg_data['file_name'])
    except FileNotFoundError:
        reg_data['data']  = pd.read_csv(local_path + reg_data['file_name'])
    else:
        print('Общая информация о датасете ', reg_data['file_name'], ':', sep='')
        reg_data['data'].info()
        print('\nОбзор данных датасета:', sep='')
        display(reg_data['data'])
        print('Минимальные значения в датасете ', reg_data['file_name'], ':', sep='')
        print(reg_data['data'].min(), '\n')
        print('Максимальные значения в датасете ', reg_data['file_name'], ':', sep='')
        print(reg_data['data'].max(), '\n')
        print('Количество дубликатов в датасете ', reg_data['file_name'], ':', sep='')
        print(reg_data['data'].duplicated().sum(), '\n')
        print('Колонки с пропущенными значениями в датасете ', reg_data['file_name'], ':\n', sep='')
        print(reg_data['data'].isna().sum(), '\n')
        
        target = reg_data['data']['product']
        features = reg_data['data'].drop(['product', 'id'], axis=1)
        reg_data['features_train'], reg_data['features_valid'], reg_data['target_train'], reg_data['target_valid'] = train_test_split(features, target, test_size = 0.25, 
                                                                                                 random_state = 12345)
        print('features_train.shape:', reg_data['features_train'].shape, round(reg_data['features_train'].shape[0]/features.shape[0], 2)*100, '% от features')
        print('features_valid.shape:', reg_data['features_valid'].shape, round(reg_data['features_valid'].shape[0]/features.shape[0], 2)*100, '% от features')
        print('target_train.shape:', reg_data['target_train'].shape, round(reg_data['target_train'].shape[0]/target.shape[0], 2)*100, '% от target')
        print('target_valid.shape:', reg_data['target_valid'].shape, round(reg_data['target_valid'].shape[0]/target.shape[0], 2)*100, '% от target')
        
        df = reg_data['features_train'].copy()
        numeric = ['f0', 'f1', 'f2']
        scaler = StandardScaler()
        scaler.fit(df[numeric])
        df[numeric] = scaler.transform(df[numeric])
        reg_data['features_train'] = df
        df = reg_data['features_valid'].copy()
        df[numeric] = scaler.transform(df[numeric])
        reg_data['features_valid'] = df
        reg_data['scaler'] = scaler
        print('________________________________________________________________________________________')

Общая информация о датасете geo_data_0.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Обзор данных датасета:


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


Минимальные значения в датасете geo_data_0.csv:
id             006OJ
f0         -1.408605
f1         -0.848218
f2        -12.088328
product          0.0
dtype: object 

Максимальные значения в датасете geo_data_0.csv:
id              zzzLH
f0           2.362331
f1           1.343769
f2           16.00379
product    185.364347
dtype: object 

Количество дубликатов в датасете geo_data_0.csv:
0 

Колонки с пропущенными значениями в датасете geo_data_0.csv:

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64 

features_train.shape: (75000, 3) 75.0 % от features
features_valid.shape: (25000, 3) 25.0 % от features
target_train.shape: (75000,) 75.0 % от target
target_valid.shape: (25000,) 25.0 % от target
________________________________________________________________________________________
Общая информация о датасете geo_data_1.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


Минимальные значения в датасете geo_data_1.csv:
id             0022J
f0        -31.609576
f1        -26.358598
f2         -0.018144
product          0.0
dtype: object 

Максимальные значения в датасете geo_data_1.csv:
id              zzzvI
f0          29.421755
f1          18.734063
f2           5.019721
product    137.945408
dtype: object 

Количество дубликатов в датасете geo_data_1.csv:
0 

Колонки с пропущенными значениями в датасете geo_data_1.csv:

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64 

features_train.shape: (75000, 3) 75.0 % от features
features_valid.shape: (25000, 3) 25.0 % от features
target_train.shape: (75000,) 75.0 % от target
target_valid.shape: (25000,) 25.0 % от target
________________________________________________________________________________________
Общая информация о датасете geo_data_2.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


Минимальные значения в датасете geo_data_2.csv:
id             009Gl
f0         -8.760004
f1          -7.08402
f2        -11.970335
product          0.0
dtype: object 

Максимальные значения в датасете geo_data_2.csv:
id              zzz9h
f0           7.238262
f1           7.844801
f2          16.739402
product    190.029838
dtype: object 

Количество дубликатов в датасете geo_data_2.csv:
0 

Колонки с пропущенными значениями в датасете geo_data_2.csv:

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64 

features_train.shape: (75000, 3) 75.0 % от features
features_valid.shape: (25000, 3) 25.0 % от features
target_train.shape: (75000,) 75.0 % от target
target_valid.shape: (25000,) 25.0 % от target
________________________________________________________________________________________


**Вывод**   
Данные загрузили в словарь, сделали выборки в соотношении 75:25, стандартизировали числовые данные столбцов features.

## Обучение и проверка модели

In [37]:
for key_name, reg_data in data_files.items():
    
    model = LinearRegression()
    model.fit(reg_data['features_train'], reg_data['target_train'])
    reg_data['predicted'] = pd.Series(model.predict(reg_data['features_valid']))
    reg_data['model'] = model
    print('Средний запас предсказанного сырья ', key_name, ': ', reg_data['predicted'].mean(), sep='')
    mse = mean_squared_error(reg_data['target_valid'], reg_data['predicted'])
    print('RMSE модели ', key_name, ': ', mse ** 0.5, sep='')
    print('-----------------------------------------------------------------')

Средний запас предсказанного сырья Регион_0: 92.59256778438035
RMSE модели Регион_0: 37.5794217150813
-----------------------------------------------------------------
Средний запас предсказанного сырья Регион_1: 68.728546895446
RMSE модели Регион_1: 0.893099286775617
-----------------------------------------------------------------
Средний запас предсказанного сырья Регион_2: 94.96504596800489
RMSE модели Регион_2: 40.02970873393434
-----------------------------------------------------------------


**Вывод**   
Обучили модели для 3-х регионов, посчитали среднеквадратичную ошибку для модели каждого региона. Увидели, что средние запасы предсказанного сырья для первого и третьего регионов сопоставимы, второй сильно уступает по этому показателю. Однако, для второго региона значение средней квадратичной ошибки модели существенно выше, чем для первой и третьей, что говорит о невысокой достоверности такого предсказания.

## Подготовка к расчёту прибыли

In [38]:
bdgt = 10e9
bhls = 200
rev_bhl = 450e3
min_debt = (bdgt / bhls) / rev_bhl
print(f"""Минимальный запас сырья в скважине для безубыточной добычи (прибыль равна 0) при расходах на разработку в 10 млд. руб. 
на 200 скажин в 1-ом регионе и при цене 1 ед. сырья 450 тыс. руб. составляет: {min_debt:.2f}""")

Минимальный запас сырья в скважине для безубыточной добычи (прибыль равна 0) при расходах на разработку в 10 млд. руб. 
на 200 скажин в 1-ом регионе и при цене 1 ед. сырья 450 тыс. руб. составляет: 111.11


**Вывод**   
Путём несложных расчётов минимального запаса сырья в скажине и сравнения полученного значения со средними по регионам, приходим к выводу, что ни в одном регионе средний запас сырья на скважину не превышает минимальный порог, в связи с чем задача выбора региона разработки новых скважин на основе машинного прогнозирования является весьма актульной при принятии управленческих решений.

## Расчёт прибыли и рисков 

In [45]:
def revenue_counter(target):
    return (target.sum() * rev_bhl) - bdgt
    
for key_name, reg_data in data_files.items():
    state = RandomState(12345)
    values = []
    
    predicted = pd.Series(reg_data['predicted']).reset_index(drop=True)
    target_valid = pd.Series(reg_data['target_valid']).reset_index(drop=True)
    for i in range(1000):
        target_subsample = target_valid.sample(n=500, replace=True, random_state=state)
        pred_subsample = predicted[target_subsample.index].sort_values(ascending=False).head(200)
        values.append(revenue_counter(target_subsample[pred_subsample.index].head(200)))
    values = pd.Series(values)
    mean = values.mean() / 1e6
    lower = values.quantile(0.025) / 1e6
    upper = values.quantile(0.975) / 1e6
    prob_los = ((values <= 0).sum() / len(values))*100
    print(f'{key_name}:')
    print(f'Средняя прибыль: {mean:.2f} млн.руб.')
    print(f'95%-й доверительный интервал: ({lower:.2f}, {upper:.2f})')
    print(f'Вероятность убытка: {prob_los:.2f}%')
    print('________________________________________________________________________________________')

Регион_0:
Средняя прибыль: 425.94 млн.руб.
95%-й доверительный интервал: (-102.09, 947.98)
Вероятность убытка: 6.00%
________________________________________________________________________________________
Регион_1:
Средняя прибыль: 515.22 млн.руб.
95%-й доверительный интервал: (68.87, 931.55)
Вероятность убытка: 1.00%
________________________________________________________________________________________
Регион_2:
Средняя прибыль: 435.01 млн.руб.
95%-й доверительный интервал: (-128.88, 969.71)
Вероятность убытка: 6.40%
________________________________________________________________________________________


**Вывод**   
На основании проведённого прогностического исследования, считаю возможным предложить для разработки 200 новых скважин Регион_0, т.к. использованная модель логистической регрессии показала на этом регионе лучшую метрику RMSE, вероятность убытка в пределах заданных задачей параметров, 95% доверительный интервал по нижней границе не имеет убыточного значения.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован